## Домашнее задание
Вы работаете продуктовым аналитиком в компании по доставке продуктов на дом. Сервис доступен как в приложении на ios, так и на android. Вы настроили фронтовую аналитику в AppMetrica, и в конце квартала маркетинг-менеджер попросил вас проанализировать поведение пользователей, а также оценить эффективность каналов их привлечения. 

Вы выгрузили данные из AppMetrica за период с 1 января по 31 марта 2020, только по пользователям, зарегистрированным позднее 1 января 2020.

Описание данных 

`date` – дата совершения события

`event` - событие

* `app_install` – установка приложения
* `app_start` – открыть приложения
* `registration` – зарегистрироваться 
* `search` – перейти на страницу поиска товаров (каталог)
* `open_item` – открыть товар
* `choose_item` – отправить товар в корзину
* `tap_basket` – перейти в корзину
* `purchase` – подтверждение покупки

`gender` – пол пользователя

`os_name` – платформа пользователя

`city` – город пользователя

`device_id` – идентификатор устройства пользователя

`urm_source` – канал, с которого пришел пользователь

* `yandex-direct` – Яндекс директ
* `google_ads` – реклама в Google
* `vk_ads` – реклама в ВК
* `instagram_ads` – реклама в instagram
* `facebook_ads` – реклама в facebook
* `referral` – акция «приведи друга»

Если стоит ‘-’, то канал не определен или это скачивание приложения напрямую или посещение не с рекламы  
`purchase_sum` – стоимость покупки (при совершении события ‘purchase’)

Обратите внимание на следующее:

- В выгрузке только уникальные действия пользователей за каждый день 

- Можно миновать стадию установки приложения, если оно было установлено ранее

- Можно миновать стадию регистрации, если пользователь был уже залогинен на момент сессии. Однако незарегистрированные пользователи не могут оформить покупку. 

## 1. Какие две метрики, по сравнению с остальными, поможет ответить на вопрос «Насколько хорошо маркетологи справляются со своей задачей привлечения новых пользователей»?

* **CAC** _**(привл. клиентов)**_
* **Количество инсталлов** 
* Retention 7-го дня _**(удержание это не привлечение)**_
* DAU _**(уже привлеченные включены)**_
* Количество сессий _**(неуник. польз-ли)**_

## 2. Какая не денежная метрика в динамике лучше всего отражает нравится ли продукт пользователям?

* CR из установки в открытие приложения
* MAU _**(ч-та польз-ния)**_
* **CR в повторную покупку в течение месяца**
<br> _(готовы платить чтобы продолжить польз-ние = неразонравился)_
* Выручка _**(денежная)**_
* CR из открытия приложения в покупку _**(еще не знают нравится ли)**_

## 3. MAU февраля:

```python
df[df.date.dt.month()==2].query('event=app_start').device_id.nunique()      
```

Ответ: `75032`

## 4. Количество установок в январе:

```python
df[df.date.dt.month()==1].query('event=app_install').device_id.nunique()     
```

Ответ: `80297`

## 5. Присвойте пользователям когорты по дню установки приложения и посчитайте для них  конверсию из установки в покупку в течение 7 дней. Для какой когорты конверсия была наибольшей? Ответ впишите в формате: дд.мм.гггг

Примечание: считаем пользователя сконвертировавшимся, если с момента установки до совершения первой покупки прошло не более 7 дней.

<br>0 цикл for day
<br>1 list of ids кто установил в день day 
<br>(#в идеале нужен чек что не удалил и установил еще раз за 7 дней
<br>-уст1 -уст2 -покуп within 7days => -уст1 исказит конверсию в свой день
<br>нужно чекнуть что нет установок подряд внутри 7 дней
<br>если нет вообще повторных установок, то это не нужно)
<br>2 isin(list of ids) **+** query 7 days **+** event=purch + id.unique()
<br>3 count/len(list)
<br>4 if > prev max => new max

```python 
for id in df.device_id.unique().to_list():
    if df.query('device_id==@id and event==app_install').count() > 1:
        print('found repeated install')
        break()

max_conv = 0
import datetime 
max_day = datetime.now()

for day in df.date.dt.date.to_list():
    id_list = df.query('date==@day and event==app_install').device_id.unique().to_list() 
    converted = df[df.device_id.isin(id_list)] \
        .query('@day <= date <= @day + pd.Timedelta(days=7) and event="purchase"'). \
        .device_id.nunique() 
    conv = converted/len(list)
    if conv > max_conv:
        max_conv == conv 
    max_day = day
print(max_day, max_conv)
```

Ответ: `01.01.2020`

## 6. Укажите значение CR из предыдущего вопроса в % (округлите до 1 цифры после запятой). Формат числа - с точкой. Пример: 66.4.

Ответ: `39.3`

## 7. С какого платного маркетингового канала пришло больше всего новых пользователей? 

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

NB! В этом и подобном ему заданиях писать надо только название, без кавычек.

```python
df.query('event==app_install').groupby('urm_source').device_id.nunique() 
``` 

Ответ: `Яндекс`

## 8. Проанализируйте на каком этапе воронки отваливается бОльшая часть клиентов. Посмотрите отдельно сценарии для зарегистрированных и для незарегистрированных пользователей. На каком шаге отваливается больше всего зарегистрированных пользователей?

В ответ впишите один из шагов, конверсия из которого (в следующий шаг) самая низкая, в таком формате: «Поиск», «Добавление товара», «Переход в корзину», «Регистрация».

```python
stages = [app_install, app_start, registration, search, open_item, choose_item, tap_basket, purchase]
event_rang = dict.fromkeys(stages, list(range(1, 9)))

df2 = df.sort_values('date')
df2['event'] = df2.event.apply(lambda x: event_rang[x])

id_reg = []
ev_reg = []
id_noreg = [] 
ev_noreg = []

for id in df.device_id.unique():
	events = df2.query('device_id==@id').event.to_list()
	if 3 in events:
		events = [ int(x) for x in (''.join(str(x) for x in events).split('3')[1]) ]
		id_reg.append(id)
		ev_reg.append(events))
	else:
		id_noreg.append(id) 
		ev_noreg.append(max(events))

df_reg = pd.DataFrame(list(zip(id_reg, ev_reg)),
               columns =['id', 'last_event'])
df_noreg = pd.DataFrame(list(zip(id_noreg, ev_noreg)),
               columns =['id', 'last_event'])

print(df_reg.last_event.value_counts(), df_noreg.last_event.value_counts())
```

Ответ: `Переход в корзину`

## 9. Пользователи, пришедшие с каких каналов, показали самую низкую конверсию в первую покупку?

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

Подсказка: 

- CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение 

```python
df_all = pd.concat([df_reg, df_noreg])
df_all = df_all.merge(df[['device_id', 'urm_source']], left_on='id', right_on='device_id')

CRA = df_all.query('last_event==6').groupby('urm_source').size()
CRB = df_all.query('last_event>=2').groupby('urm_source').size()
CR = CRA/CRB 
``` 

Ответ: `Яндекс`

## 10. Пользователи, пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки пользователей)

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

```python
dff = df.query('event=="purchase"').groupby(device_id).first().purchase_sum
dff.groupby('urm_source').purchase_sum.median()
``` 

Ответ: `Реферальная программа`

## 11. Данные по затратам на рекламу:

Яндекс – 10 491 707 руб.

Гугл – 10 534 878 руб.

Фейсбук – 8 590 498 руб.

Инстаграм – 8 561626 руб.

ВК – 9 553 531руб.

Расходы на реферальную программу: если пользователь приведет друга и последний совершит первую покупку, то оба получат по 100 рублей.

Какой платный канал привлечения имеет самый высокий ROMI? 

![](https://www.carrotquest.io/blog/wp-content/uploads/2020/10/image-2-1100x163.png)

В ответ впишите один из каналов в таком формате: «Яндекс», «Гугл», «ВК», «Фейсбук», «Инстаграм», «Реферальная программа».

```python
#расходы реферальной программы
df_all.query('urm_source=="referral" and last_event==6').count()*200

#доходы
df.groupby('urm_source').purchase_sum.sum()
``` 

Ответ: `ВК`